In [1]:
import sys, unittest, time, datetime
import urllib.request, urllib.error, urllib.parse
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import InvalidArgumentException

import pandas as pd
import numpy as np
import time
import re
# import logging
# logging.basicConfig(filename='YT_scraping_log.log',level=logging.DEBUG,format='%(asctime)s.%(msecs)03d %(levelname)s %(module)s - %(funcName)s: %(message)s',datefmt='%Y-%m-%d %H:%M',)

In [2]:
import os

In [3]:
os.listdir()

['.git',
 '.idea',
 '.ipynb_checkpoints',
 '01_get_YT_links.ipynb',
 '01_scrape_youtube_metadata.py',
 'GMAT Sentence correction_youtube_scrape_metadata.csv',
 'raag malkauns_youtube_scrape_metadata.csv',
 'sample_audio.m4a',
 'sample_audio2.m4a',
 'scrapeYT.py',
 'selenium_based_scraper.py',
 'selenium_based_scrapper2.py',
 'temp_data_extract.csv',
 'Untitled.ipynb',
 'Youtube_scraping.ipynb',
 'yTdownload',
 'YT_scraping_log.log']

In [4]:
from pydub import AudioSegment
audio = AudioSegment.from_file('sample_audio2.m4a')

In [19]:
AudioSegment.apply_mono_filter_to_each_channel

In [13]:
audio.export('try.mp3',format = 'mp3')

<_io.BufferedRandom name='try.mp3'>

In [15]:
audio[1000:1200]

In [6]:
print("Information:")
print("Channels:", audio.channels)
print("Bits per sample:", audio.sample_width * 8)
print("Sampling frequency:", audio.frame_rate)
print("Length:", audio.duration_seconds, "seconds")

Information:
Channels: 2
Bits per sample: 16
Sampling frequency: 44100
Length: 259.1114739229025 seconds


In [7]:
freqs, times, amplitudes = audio.spectrogram(window_length_s=0.03, overlap=0.5)
amplitudes = 10 * np.log10(amplitudes + 1e-9)
# Plot
plt.pcolormesh(times, freqs, amplitudes)
plt.xlabel("Time in Seconds")
plt.ylabel("Frequency in Hz")
plt.show()

AttributeError: 'AudioSegment' object has no attribute 'spectrogram'

In [ ]:
def scrape_metadata_from_youtube(textToSearch = 'Ahir Bhairav',maxheight=15000):
    textToSearch  = textToSearch.replace(" ","+")
    print("Quering ",textToSearch)
    url = "https://www.youtube.com/results?search_query={}&sp=EgIQAQ%253D%253D".format(textToSearch)
    print("URL to fetch: ",url)
    t0 = time.time()
    driver=webdriver.Chrome()
    driver.get(url)
    height = driver.execute_script("return document.documentElement.scrollHeight")
    lastheight = 0
    while True:
        if lastheight > maxheight:
            break
        lastheight = height
        driver.execute_script("window.scrollTo(0, " + str(height) + ");")
        time.sleep(4)
        height = driver.execute_script("return document.documentElement.scrollHeight")
    
    user_data = driver.find_elements_by_xpath('//*[@id="video-title"]')
    channel_data = driver.find_elements_by_xpath('//*[@id="text"]/a')
    channel_list = [x.text for x in channel_data][1::2]
    
    details_list = [i.get_attribute('aria-label') for i in user_data]
    url_list = [i.get_attribute('href') for i in user_data]
    title_list = [i.get_attribute('title') for i in user_data]
    logging.debug(f'Search Key: {textToSearch}')
    logging.debug('Total Time taken:{}s'.format(time.time() - t0))
    logging.debug('No. of Records scraped: ', len(user_data))
    data = prepare_dataframe(title_list , details_list , url_list,channel_list)
    return data , [title_list , details_list , url_list, channel_list]

In [ ]:
def prepare_dataframe(a,b,c,d):
    df = pd.DataFrame({'urls':c,'title':a,'details':b, 'channel':d})
    #df['duration'] = df['details'].str.extract('([0-9]* hour{1}s{0,1}| [0-9]* hour{1}s{0,1}, [0-9]* minute{1}s{0,1} |[0-9]* minute{1}s{0,1})')
    df['ytkey'] = df['urls'].str[-11:]
    df['hours'] = df['details'].str.extract('([0-9]*) hour{1}s{0,1}').fillna(0).astype('int')
    df['minutes'] = df['details'].str.extract('([0-9]*) minute{1}s{0,1}').fillna(0).astype('int')
    df['views'] = df['details'].str.extract('([0-9]*,*[0-9]*) views')
    df['views'] = df['views'].str.replace(',',"").astype('int')
    df['age_years']=df['details'].str.extract('([0-9]*) year{1}s{0,1} ago').fillna(0).astype('int')
    df['age_months']=df['details'].str.extract('([0-9]*) month{1}s{0,1} ago').fillna(0).astype('int')
    return df

In [ ]:
data , details = scrape_metadata_from_youtube('raag kedar',5000)
# data.to_csv('temp_data_extract.csv',index = False)

In [ ]:
data.head()

In [ ]:
data.to_csv('temp_data_extract.csv',index = False)

## Test : Ignore


In [ ]:
user_data[0].get_attribute('aria-label')

In [ ]:
df_meta = pd.DataFrame()

In [ ]:
details_list = [i.get_attribute('aria-label') for i in user_data]
url_list = [i.get_attribute('href') for i in user_data]

In [ ]:
for i in user_data:
    print(i.get_attribute('aria-label'))
    print(i.get_attribute('href'))
#     link = (i.get_attribute('href'))

In [ ]:
for i in user_data:
    print(i.get_attribute('title'))
#     link = (i.get_attribute('href'))

In [ ]:
channel_data = driver.find_elements_by_xpath('//*[@id="text"]/a')

In [ ]:
for p in channel_data:
    print(p.text)

In [ ]:
driver.bfind_element_by_class_name('style-scope ytd-video-meta-block').text

In [ ]:
driver.find_element_by_tag_name('span').text

In [ ]:
views_data = driver.find_element_by_xpath('//*[@id="channel-name"]')

In [ ]:
views_data

In [ ]:
for p in views_data:
    print(p.text)